In [1]:
import tensorflow as tf
import os
from keras.layers import Conv2D, BatchNormalization, Dense, ReLU, GlobalAveragePooling2D
from keras import optimizers, Model
from keras.callbacks import ModelCheckpoint
from keras.callbacks import CSVLogger
from keras.utils import plot_model
import tensorflow_addons as tfa
from keras import Input
import time
import tensorflow as tf

AUTOTUNE = tf.data.experimental.AUTOTUNE

In [2]:
os.environ['CUDA_DEVICE_ORDER'] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    #Invalid device or can't modify virtual devices once installed
    print("Failed to limit the GPU Memory")

In [4]:
IMAGENET_TRAINSET_SIZE = 310284
IMG_SIZE = 224
batch_size = 32
epochs = 10
initial_lr = 1e-3
ADAM_LEARNING_RATE = 0.1
lr = 0.1
logdir = "logs_224x224"
csv_logger = CSVLogger('log.csv', separator=',', append=True)
adam = optimizers.Adam(learning_rate=ADAM_LEARNING_RATE)


train_dir = r'W:/10_percent/train/'
test_dir = r'W:/10_percent/test/'

In [5]:
# def read_image_label(file_path):
#     label = tf.strings.split(file_path, os.path.sep)[-2]
#     label = tf.strings.to_number(label, out_type=tf.int32)
#     image = tf.io.read_file(file_path)
#     image = tf.image.decode_jpeg(image, channels=3)
#     return image, label

def read_image_label(file_path):
    label = tf.strings.split(file_path, os.path.sep)[-2]
    label = tf.strings.to_number(label, out_type=tf.int32)
    label = tf.one_hot(label, 8631)
    image = tf.io.read_file(file_path)
    image = tf.image.decode_jpeg(image, channels=3)
    return image, label



def train_transform(image, label):
    image = tf.cast(image, tf.float32)
    image = tf.image.resize_with_crop_or_pad(image, IMG_SIZE + 6, IMG_SIZE + 6)
    image = tf.image.random_crop(image, size=[IMG_SIZE, IMG_SIZE, 3])
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tfa.image.rotate(image, tf.random.uniform(shape=(), maxval=1))
    image = tf.image.random_brightness(image, 0.2)
    image = tf.math.divide(image, 255.0)
    image = (image - tf.constant([0.485, 0.456, 0.406])) / tf.constant(
        [0.229, 0.224, 0.225]
    )
    return image, label


def test_transform(image, label):
    image = tf.cast(image, tf.float32)
    image = tf.math.divide(image, 255.0)
    image = (image - tf.constant([0.485, 0.456, 0.406])) / tf.constant(
        [0.229, 0.224, 0.225]
    )
    return image, label

In [6]:
train_ds = tf.data.Dataset.list_files(os.path.join(train_dir, "*", "*.jpg"))
train_ds = train_ds.map(read_image_label, num_parallel_calls=AUTOTUNE)
train_ds = train_ds.shuffle(buffer_size=10000)
train_ds = train_ds.map(train_transform, num_parallel_calls=AUTOTUNE)
train_ds = train_ds.batch(batch_size)
train_ds = train_ds.prefetch(AUTOTUNE)

test_ds = tf.data.Dataset.list_files(os.path.join(test_dir, "*", "*.jpg"))
test_ds = test_ds.map(read_image_label, num_parallel_calls=AUTOTUNE)
test_ds = test_ds.map(test_transform, num_parallel_calls=AUTOTUNE)
test_ds = test_ds.batch(batch_size)
test_ds = test_ds.prefetch(AUTOTUNE)

In [7]:
for images, labels in train_ds.take(1):
    print(labels.shape)

(32, 8631)


In [8]:
def repvgg_block(inputs, out_channels, strides, stage, block):
    """
    RepVGG Block
    """
    if strides == 1:
        identity = inputs
    else:
        identity = Conv2D(out_channels, kernel_size=3, strides=strides, padding='same', use_bias=True, name=stage+"_idconv_"+block)(inputs)
        identity = BatchNormalization(name=stage+"_idbn_"+block)(identity)

    x = Conv2D(out_channels, kernel_size=3, strides=strides, padding='same', use_bias=True, name=stage+"_conv_"+block)(inputs)
    x = BatchNormalization(name=stage+"_bn_"+block)(x)
    x = ReLU()(x)

    x = Conv2D(out_channels, kernel_size=3, strides=1, padding='same', use_bias=True, name=stage+"_conv_"+block+"_1")(x)
    x = BatchNormalization(name=stage+"_bn_"+block+"_1")(x)
    x = ReLU()(x)

    x = Conv2D(out_channels, kernel_size=3, strides=1, padding='same', use_bias=True, name=stage+"_conv_"+block+"_2")(x)
    x = BatchNormalization(name=stage+"_bn_"+block+"_2")(x)

    x = x + identity
    x = ReLU()(x)

    return x

In [9]:
def create_RepVGG_A0(input_shape=(224, 224, 3), num_classes=8631):
    
    # Define input layer
    inputs = Input(shape=input_shape)

    x = repvgg_block(inputs, 64, 2, "stage1", "block1")
    x = repvgg_block(x, 64, 2, "stage1", "block2")
    x = repvgg_block(x, 64, 2, "stage1", "block3")
    x = repvgg_block(x, 128, 2, "stage2", "block1")
    x = repvgg_block(x, 128, 2, "stage2", "block2")
    x = repvgg_block(x, 128, 2, "stage2", "block3")
    x = repvgg_block(x, 256, 2, "stage3", "block1")
    x = repvgg_block(x, 256, 2, "stage3", "block2")
    x = repvgg_block(x, 256, 2, "stage3", "block3")
    x = repvgg_block(x, 512, 2, "stage4", "block1")
    x = repvgg_block(x, 512, 2, "stage4", "block2")
    x = repvgg_block(x, 512, 2, "stage4", "block3")

    # Classifier
    x = GlobalAveragePooling2D()(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    # Create model
    model = Model(inputs=inputs, outputs=outputs)
    
    return model

In [10]:
model = create_RepVGG_A0()
plot_model(model, to_file='model_architecture2.png', show_shapes=True)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 stage1_conv_block1 (Conv2D)    (None, 112, 112, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 stage1_bn_block1 (BatchNormali  (None, 112, 112, 64  256        ['stage1_conv_block1[0][0]']     
 zation)                        )                                                             

## Checkpoint creation (in order to train the model in different days)

In [11]:
checkpoint_dir = './model_checkpoint'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

In [12]:
# Create a ModelCheckpoint callback to save the model weights
# every 10 epochs and only save the best weights based on 
# validation loss
checkpoint_filepath = './model_checkpoint'

# Define callbacks to save checkpoints every 10 epochs
checkpoint_filepath = checkpoint_filepath
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    save_freq=10*len(train_ds),  # Save every 10 epochs
)

In [13]:
# Set learning rate
initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)

In [14]:
# Compile model
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    metrics=['accuracy']
)

In [15]:
history = model.fit(train_ds,
                    validation_data=test_ds,
                    epochs=3,
                    callbacks=[checkpoint_callback, csv_logger])

Epoch 1/3
2859/9697 [=======>......................] - ETA: 15:10 - loss: 8.4671 - accuracy: 1.8582e-04

KeyboardInterrupt: 

In [ ]:
model.save_weights('3e_normDS_weights.h5')
model.save('final_model.h5')

In [ ]:
# check time for evaluation
start_ts_eval = time.time()
model.evaluate(test_ds)
print(f"Evaluation of trained model takes: {time.time() - start_ts_eval:.4f}s")

In [ ]:
latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
model.load_weights(latest_checkpoint)

In [ ]:
from keras.models import load_model

# load the pre-trained model
model = load_model('final_model.h5')

In [ ]:

history = model.fit(train_ds,
                    validation_data=test_ds,
                    epochs=10,
                    callbacks=[tf.keras.callbacks.TensorBoard(log_dir=logdir, profile_batch=0),])

In [ ]:
model.save_weights('30e+10_normDS_weights.h5')
model.save('final_model.h5')